In [5]:
import numpy as np

In [6]:
a = [[1,2,3,4],[5,6,7,8]]
a = np.array(a)
new_shape = (4,2)

b = np.reshape(a, new_shape)

print("a" , a)
print("b" , b)


a [[1 2 3 4]
 [5 6 7 8]]
b [[1 2]
 [3 4]
 [5 6]
 [7 8]]


In [ ]:
# reshape vs transpose
# reshape 只改变形状，不改变元素顺序；transpose 会交换轴：
print("reshape(4,2):\n", a.reshape(4, 2))
print("transpose:\n", a.T)

reshape(4,2):
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
transpose:
 [[1 5]
 [2 6]
 [3 7]
 [4 8]]


In [13]:
def is_invertible(a):
    if len(a) != len(a[0]):
        return false
    return np.linalg.det(a) != 0

def transform_matrix(A, T, S):
    if not is_invertible(T) or not is_invertible(S) :
        print("T or S is not invertible")
        return -1
    else:
        transformed_matrix = (np.linalg.inv(T) @ A) @ S
    return transformed_matrix

print(transform_matrix([[1, 2], [3, 4]], [[2, 0], [0, 2]], [[1, 1], [0, 1]]))

[[0.5 1.5]
 [1.5 3.5]]


In [3]:
features = [[0.5, 1.0], [-1.5, -2.0], [2.0, 1.5]]
labels = [0, 1, 0]
weights = [0.7, -0.4]
bias = -0.1

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
prob =sigmoid(np.dot(features,weights) + bias)
print(prob)
mse = np.round(np.mean((labels - prob) **2), 4)
print(mse)

[0.46257015 0.41338242 0.66818777]
0.3349


In [1]:
import numpy as np

In [18]:
features = [[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]
labels = [1, 0, 0]
initial_weights = [0.1, -0.2]
initial_bias = 0.0
learning_rate = 0.1
epochs = 2

In [ ]:
features = np.array(features)
labels = np.array(labels)
weights = np.array(initial_weights)
bias = initial_bias
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)
n = len(labels)
mse_loss = []
for epoch in range(epochs):
    # foward
    z = np.dot(features,weights) + bias
    predict = sigmoid(z)
    # mse loss
    loss = np.mean((np.array(labels) - predict) ** 2)
    mse_loss.append(loss)
    #gradient descent
    dloss_dpred = 2 * (predict - labels) / n   # n
    dpred_dz = sigmoid_derivative(predict)     # n
    dz_dw = features                           # n * m
    dz_db = 1

    print(dloss_dpred)
    print(dpred_dz)
    grad_z = dloss_dpred * dpred_dz  # n * n =  n
    print(grad_z)
    grad_w = np.dot(grad_z, dz_dw)  # n * m
    print("grad_w",grad_w)
    grad_b = np.sum(grad_z) * dz_db 
    print("grad_b", grad_b)
    # update weights and bias
    weights -= learning_rate*grad_w
    bias -= learning_rate*grad_b

print("weights", weights)
print("bias", bias)
print("mse_loss", mse_loss)

[-0.38296168  0.33333333  0.38296168]
[0.24445831 0.25       0.24445831]
[-0.09361817  0.08333333  0.09361817]
grad_w [-0.02056966 -0.29113933]
grad_b 0.08333333333333334
[-0.37446407  0.33748221  0.37172584]
[0.24619359 0.24996127 0.24668354]
[-0.09219065  0.08435748  0.09169864]
grad_w [-0.01517434 -0.28342112]
grad_b 0.08386547041703868
weights [ 0.1035744  -0.14254396]
bias -0.016719880375037202
mse_loss [0.3033228034139421, 0.2942232621822798]


In [8]:
class Value:
	def __init__(self, data, _children=(), _op=''):
		self.data = data
		self.grad = 0
		self._backward = lambda: None
		self._prev = set(_children)
		self._op = _op
	def __repr__(self):
		return f"Value(data={self.data}, grad={self.grad}, prev={self._prev}, op={self._op})"

	def __add__(self, other):
		# Implement addition here
		other = other if isinstance(other, Value) else Value(other)
		out = Value(self.data + other.data, (self, other), '+')
		def _backward():
			self.grad +=out.grad
			other.grad += out.grad
		out._backward = _backward
		return out

	def __mul__(self, other):
		# Implement multiplication here
		other = other if isinstance(other, Value) else Value(other)
		out = Value(self.data * other.data, (self, other), '*')
		def _backward():
			self.grad += other.data * out.grad
			other.grad += self.data * out.grad
		out._backward = _backward
		return out

	def relu(self):
		out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

		def _backward():
			self.grad += (out.data > 0) * out.grad
		out._backward = _backward

		return out

	def backward(self):
		# topological order all of the children in the graph
		topo = []
		visited = set()
		def build_topo(v):
			if v not in visited:
				visited.add(v)
				for child in v._prev:
					build_topo(child)
				topo.append(v)
		build_topo(self)

		# go one variable at a time and apply the chain rule to get its gradient
		self.grad = 1
		for v in reversed(topo):
			v._backward()

In [9]:
a = Value(2.0)
b = Value(3.0)
c = a * b


c.backward()

print("a:", a)
print("b:", b) 
print("c:", c)

a = Value(2.0)
b = Value(3.0)
d = a + b
d.backward()
print("a:", a)
print("b:", b) 
print("d:", d)

a: Value(data=2.0, grad=3.0, prev=set(), op=)
b: Value(data=3.0, grad=2.0, prev=set(), op=)
c: Value(data=6.0, grad=1, prev={Value(data=3.0, grad=2.0, prev=set(), op=), Value(data=2.0, grad=3.0, prev=set(), op=)}, op=*)
a: Value(data=2.0, grad=1, prev=set(), op=)
b: Value(data=3.0, grad=1, prev=set(), op=)
d: Value(data=5.0, grad=1, prev={Value(data=3.0, grad=1, prev=set(), op=), Value(data=2.0, grad=1, prev=set(), op=)}, op=+)


In [34]:

import numpy as np
import copy
import math

# DO NOT CHANGE SEED
np.random.seed(42)

# DO NOT CHANGE LAYER CLASS
class Layer(object):

	def set_input_shape(self, shape):
	
		self.input_shape = shape

	def layer_name(self):
		return self.__class__.__name__

	def parameters(self):
		return 0

	def forward_pass(self, X, training):
		raise NotImplementedError()

	def backward_pass(self, accum_grad):
		raise NotImplementedError()

	def output_shape(self):
		raise NotImplementedError()

# Your task is to implement the Dense class based on the above structure
class Dense(Layer):
	def __init__(self, n_units, input_shape=None):
		self.layer_input = None
		self.input_shape = input_shape
		self.n_units = n_units
		self.trainable = True
		self.W = None
		self.w0 = None
		self.optimizer_w = None
		self.optimizer_w0 = None
	
	def initialize(self, optimizer):
		limit  = 1 / np.sqrt(self.input_shape)
		self.W = np.random.uniform(-limit, limit, (self.input_shape[0], self.n_units))
		self.w0 = np.zeros((1,self.n_units))
		self.optimizer_w = optimizer
		self.optimizer_w0 = optimizer

	def forward_pass(self,x):
		self.x = x
		return np.dot(x, self.W) + self.w0

	def backward_pass(self, accum_grad):
		grad_w = np.dot(self.x.T, accum_grad)
		print("grad_w:", grad_w)
		grad_b = np.sum(accum_grad, axis=0, keepdims =True)
		print("grad_b:", grad_b)
		grad_input = np.dot(accum_grad, self.W.T)
		print("grad_input:", grad_input)
		self.optimizer_w.update(self.W,grad_w)
		self.optimizer_w0.update(self.w0,grad_b)
		return grad_input

	def number_of_parameters(self):
		return np.prod(self.W.shape) + np.prod(self.w0.shape),self.W.shape,self.w0.shape
	
	def output_shape(self):
		return (self.input_shape, self.n_units)

	

In [35]:
# Initialize a Dense layer with 3 neurons and input shape (2,)
dense_layer = Dense(n_units=3, input_shape=(2,))

# Define a mock optimizer with a simple update rule
class MockOptimizer:
    def update(self, weights, grad):
        return weights - 0.01 * grad

optimizer = MockOptimizer()

# Initialize the Dense layer with the mock optimizer
dense_layer.initialize(optimizer)


In [36]:
# Perform a forward pass with sample input data
X = np.array([[1, 2]])
output = dense_layer.forward_pass(X)
print("Forward pass output:", output)

# Perform a backward pass with sample gradient
accum_grad = np.array([[0.1, 0.2, 0.3]])
back_output = dense_layer.backward_pass(accum_grad)
print("Backward pass output:", back_output)

Forward pass output: [[ 0.10162127 -0.33551992 -0.64490545]]
grad_w: [[0.1 0.2 0.3]
 [0.2 0.4 0.6]]
grad_b: [[0.1 0.2 0.3]]
grad_input: [[ 0.20816524 -0.22928937]]
Backward pass output: [[ 0.20816524 -0.22928937]]


In [33]:
input_shape=(2,)
input_shape[0]
para, w, w0 = dense_layer.number_of_parameters()
print(w)
print(w0)

(2, 3)
(1, 3)


In [44]:
X = np.array([[1, 2]])
y = np.array([1, 2])
z = X.T
print(X.shape)
print(y.shape)
print(z.shape)
print(z)
x = np.array([1.0, 2.0])
print(x.shape)
w1 = np.array([[1.0, 0.0], [0.0, 1.0]])
print(w1.shape)

(1, 2)
(2,)
(2, 1)
[[1]
 [2]]
(2,)
(2, 2)


In [40]:
s = 6
a = 2
c = s / a
print(c)

3.0


In [43]:
import numpy as np
import math
def compute_cross_entropy_loss(predicted_probs: np.ndarray, true_labels: np.ndarray, epsilon = 1e-15) -> float:
    n = len(true_labels)
    loss = 0.0
    for i in range(n):
        for j in range(len(true_labels[i])):
            if true_labels[i][j] == 1:
                loss -= math.log(predicted_probs[i][j] + 1e-12)
    if loss / n == -0.0:
        return 0.0
    return loss / n
pred = np.array([[1, 0, 0], [0, 1, 0]])
true = np.array([[1, 0, 0], [0, 1, 0]])
print(compute_cross_entropy_loss(pred, true))
print(round(compute_cross_entropy_loss(pred, true), 4))

-1.000088900581841e-12
-0.0


In [50]:
x = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
print(x.shape)
print(np.mean(x, axis=(0,1)))
print(np.mean(x))
print(np.mean(x, axis=(0)))

(2, 2, 3)
[5.5 6.5 7.5]
6.5
[[4. 5. 6.]
 [7. 8. 9.]]
